In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# !pip3 install tensorflow_text

****Importing Modules****

In [ ]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt   
import tensorflow as tf 
from tensorflow import keras
import tensorflow_hub as hub 
import tensorflow_text as text
import spacy

In [ ]:
train = pd.read_csv('../input/nlp-getting-started/train.csv')
test = pd.read_csv('../input/nlp-getting-started/test.csv')
train.head()

In [ ]:
train.isna().sum()

In [ ]:
from pandas_profiling import ProfileReport
profile = ProfileReport(train, title='Panads Profile Report')
train.profile_report()

In [ ]:
train_df = train.drop(['id','keyword','location'], axis=1)
test_df = test.drop(['id','keyword','location'], axis=1)

In [ ]:
test_df.head()

Introducing Bert Models

In [ ]:
# Bert Model Urls
preprocess_url = 'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3'
encoder_url = 'https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4'
# Initializing our bert models 
bert_preprocess_model = hub.KerasLayer(preprocess_url)
bert_model = hub.KerasLayer(encoder_url)

In [ ]:
X = train_df['text']
y = train_df['target']

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)


In [ ]:
# BERT LAYERS
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
preprocess_text = bert_preprocess_model(text_input)
outputs = bert_model(preprocess_text)

# Neural network layers
l = tf.keras.layers.Dropout(0.1, name="dropout")(outputs['pooled_output'])
l = tf.keras.layers.Dense(1, activation='sigmoid', name='output')(l)

# Use inputs and outputs to construct a final model
model = tf.keras.Model(inputs=[text_input], outputs=[l])

In [ ]:
model.summary()

In [ ]:
model.compile(
    optimizer = 'adam', 
    loss = 'binary_crossentropy', 
    metrics = ['accuracy'])

In [ ]:
history = model.fit(
    X_train, y_train, 
    epochs = 3,  
    validation_split = 0.1, 
)

In [ ]:
history

In [ ]:
model.evaluate(X_test, y_test)

In [ ]:
y_predicted = np.argmax(model.predict(X_test))

In [ ]:
#Create a  DataFrame with the passengers ids and our prediction regarding whether they survived or not
submission = pd.DataFrame({'id':test.index,'target':predictions})

#Visualize the first 5 rows
submission.head()